# Análisis de sentimientos

En esta sección se realiza un análisis de sentimiento de tweets, como un ejemplo del uso de modelos de lenguaje previamente entrenado. En este caso, se utiliza el modelo [Distilbert](https://arxiv.org/abs/1910.01108) para el análisis de sentimientos. Este modelo es una versión más ligera del modelo. [BERT](https://arxiv.org/abs/1810.04805), que es un modelo de lenguaje previo a la entrada que se ha utilizado con muy buenos resultados en diferentes tareas de procesamiento del lenguaje natural, como [Análisis de sentimientos](https://arxiv.org/abs/1905.05583), [Clasificación de texto](https://arxiv.org/abs/1904.09077) o [Extracción de información](https://arxiv.org/abs/1906.05237).

En este caso, se utiliza el modelo previamente entrenado para el análisis de sentimientos en inglés.

## Carga del conjunto de datos`

Usaremos la librería [datasets](https://huggingface.co/docs/datasets/) para cargar el `dataset` de los tweets. Esta La librería te permite cargar `datasets` de diferentes fuentes como [Hugging Face Hub](https://huggingface.co/datasets), [Amazon AWS](https://docs.aws.amazon.com/es_es/marketplace/latest/userguide/datasets.html) o [Google Cloud](https://cloud.google.com/ai-platform/training/docs/datasets). En este caso, cargaremos el `dataset` de tuits desde [Hugging Face Hub](https://huggingface.co/datasets/dair-ai/emotion).


In [3]:
# Instalamos las librerias que vamos a usar

import os
os.environ["WANDB_DISABLED"] = "true"

%pip install -U transformers datasets evaluate accelerate scikit-learn accuracy

  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metad

In [4]:
import datasets

# Cargamos el Dataset
dataset = datasets.load_dataset('dair-ai/emotion')

# Mostramos los datos de ejemplo

dataset['train'][0]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'text': 'i didnt feel humiliated', 'label': 0}

Podemos ver que cada registro del `dataset` Contiene el texto del tweet y el sentimiento asociado. En este caso, el sentimiento está codificado con un entero entre 0 y 5, donde 0 corresponde a la tristeza, 1 a Alegria, 2 para amor, 3 a la ira, 4 para miedo y 5 para sorpresa.

## Preparación del `dataset`

En este caso, el `dataset` Ya está dividido en conjuntos de entrenamiento, prueba y validación. El siguiente paso es preparar el `dataset` para el entrenamiento modelo. En este caso, el modelo que utilizaremos es el modelo [BERT](https://arxiv.org/abs/1810.04805). Este modelo requiere que el texto sea tokenizado y los tokens estén codificados con sus identificadores numéricos correspondientes. Para hacer esto, utilizaremos un tokenizador DistilBERT previamente entrenado.

In [5]:
# importem el tokenitzador de DistilBERT
from transformers import AutoTokenizer

# Carreguem el tokenitzador
tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased')

# Mostrem un exemple de tokenització
tokenizer.tokenize('FC Barcelona is fucked this year')

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

['▁fc', '▁barcelona', '▁is', '▁fuck', 'ed', '▁this', '▁year']

In [6]:
# Definimos una función para preprocesar el texto.
# Truncamos los textos para asegurarnos de que no excedan el máximo tamaño de entrada deDistilBert

def tokenize(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True)

Para aplicar la tokenización, usaremos la función `map` de `datasets`. Esta función permite aplicar una función a cada registro del `dataset`. En este caso, la función que aplicaremos es la función `tokenize` que hemos definido anteriormente. Nosotros también usaremos `batched=True` para indicar que la función se aplicará a todo `dataset` en bloques.

In [7]:
dades_tokenitzades = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

## Evaluación

Para evaluar el modelo, debemos cargar el método que utilizaremos para la evaluación.En este caso usaremos la métrica `accuracy` del módulo `evaluate` de HuggingFace.

También definiremos una función para calcular las métricas del modelo. Esta función se utilizará para evaluar el modelo después de cada época.

In [8]:
import evaluate

accuracy = evaluate.load('accuracy')

In [9]:
# Definimos una función para calcular la precisión del modelo

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

## Definición de etiquetas

Antes de entrenar el modelo, debemos crear un diccionario que traduzca los identificadores numéricos del sentimiento a sus etiquetas correspondientes y viceversa.

In [10]:
id_a_etiqueta = {
    0: "SADNESS",
    1: "JOY",
    2: "LOVE",
    3: "ANGER",
    4: "FEAR",
    5: "SUPRISE"
}

etiqueta_a_id = {
    "SADNESS": 0,
    "JOY": 1,
    "LOVE": 2,
    "ANGER": 3,
    "FEAR": 4,
    "SUPRISE": 5
}

## Fine tuning del modelo

El proceso de ajuste fino del modelo es entrenar el modelo con nuestro `dataset`. Esto permite que el modelo se adapte mejor a nuestros datos y mejore su rendimiento.

Necesitamos definir la función de optimización, el tamaño de los bloques y el número de épocas.

In [11]:
BATCH_SIZE = 16
NUM_EPOCHS = 1

Ahora podemos cargar el modelo previamente capacitado y hacer el ajuste fino. Usaremos `TFAutoModelForSequenceClassification` y agregaremos las etiquetas que hemos definido previamente.

In [12]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=len(etiqueta_a_id),
    id2label=id_a_etiqueta,
    label2id=etiqueta_a_id
)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Para evaluar el modelo debemos definir un objeto `TrainingArguments` con los parámetros de entrenamiento. Podemos incluir el número de épocas, el tamaño de los bloques, el tamaño del lote, la tasa de aprendizaje, etc.

In [16]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test_trainer",
    eval_strategy="epoch",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Ahora podemos entrenar el modelo, utilizando el `trainer`.

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dades_tokenitzades['train'],
    eval_dataset=dades_tokenitzades['validation'],
    compute_metrics=compute_metrics,
)

trainer.train()

AttributeError: 'TFDistilBertForSequenceClassification' object has no attribute 'to'

## Inferencia

Para influir en el modelo, crearemos una pipeline Huggingface. Esta pipeline utilizará el modelo y el tokenizador que importamos anteriormente.

Luego usaremos la pipeline para hacer una inferencia con un texto de ejemplo.

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
print(classifier("FC Barcelona is fucked this year"))
print(classifier("I'm not going to watch soccer again"))
print(classifier("Real Madrid is not going to treble this year. I'm relieved!"))